In [3]:
import os
import boto3
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_leaflet as dl
import dash_leaflet.express as dlx

In [4]:
# Fetch AWS credentials from environment variables
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_S3_ENDPOINT = os.getenv('AWS_S3_ENDPOINT')
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')

# Initialize S3 client
session = boto3.Session()
s3_client = session.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    endpoint_url=AWS_S3_ENDPOINT,
    region_name=AWS_DEFAULT_REGION
)

bucket_name = 'wamsi-westport-project-1-1'

In [5]:
def find_s3_key(agency, program, id_number):
    # Define base directory
    base_dir = f'csiem-data/data-lake/{agency}/{program}/'
    
    # List objects in the directory
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=base_dir)
    
    # Search for a file that contains the id_number
    for content in response.get('Contents', []):
        key = content['Key']
        if id_number in key:
            return key
    
    return None

In [4]:
# elif dataset_type == "ghrsst":
#             if aoi_type == 'point':
#                 coordinate = point_coordinate
#                 if coordinate is None:
#                     print("Point coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/NASA/GHRSST/Points/ghrsst_sst_point_{coordinate}.csv'
#                 title = f'{variable} Data for Point {coordinate}'
#             elif aoi_type == 'polygon':
#                 coordinate = polygon_coordinate
#                 if coordinate is None:
#                     print("Polygon coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/NASA/GHRSST/Polygon_offshore/ghrsst_sst_polygon_{coordinate}.csv'
#                 title = f'{variable} Data for Polygon {coordinate}'
#             else:
#                 print("Invalid AOI type")
#                 return {}

#         elif dataset_type == "reflectance":
#             if aoi_type == 'point':
#                 coordinate = point_coordinate
#                 if coordinate is None:
#                     print("Point coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/ESA/GlobColor/Reflectance/Point/CMEMS_reflectance_point_{coordinate}.csv'
#                 title = f'{variable} Data for Point {coordinate}'
#             elif aoi_type == 'polygon':
#                 coordinate = polygon_coordinate
#                 if coordinate is None:
#                     print("Polygon coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/ESA/GlobColor/Reflectance/Polygon/CMEMS_reflectance_polygon_{coordinate}.csv'
#                 title = f'{variable} Data for Polygon {coordinate}'
#             else:
#                 print("Invalid AOI type")
#                 return {}
            
#         elif dataset_type == "plankton":
#             if aoi_type == 'point':
#                 coordinate = point_coordinate
#                 if coordinate is None:
#                     print("Point coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/ESA/GlobColor/Plankton/Points/CMEMS_planktons_point_{coordinate}.csv'
#                 title = f'{variable} Data for Point {coordinate}'
#             elif aoi_type == 'polygon':
#                 coordinate = polygon_coordinate
#                 if coordinate is None:
#                     print("Polygon coordinate is None")
#                     return {}
#                 s3_key = f'csiem-data/data-lake/ESA/GlobColor/Plankton/Polygon/CMEMS_planktons_polygon_{coordinate}.csv'
#                 title = f'{variable} Data for Polygon {coordinate}'
#             else:
#                 print("Invalid AOI type")
#                 return {}

#         else:
#             print("Invalid dataset type")
#             return {}


,Points,longitude,latitude,label
0,1,115.6490,-31.7690,NaN
1,2,115.4790,-31.8520,NaN
2,3,115.6850,-31.9390,NaN
3,4,115.6850,-31.8960,NaN
4,5,115.3670,-32.0270,NaN
5,6,115.6860,-32.1000,NaN
6,7,115.3510,-32.1930,NaN
7,8,115.5210,-32.1880,NaN
8,9,115.7310,-32.1940,NaN
9,10,115.4410,-32.3550,NaN


In [1]:
from dash import dcc, html, Input, Output, State, callback
import geopandas as gpd
import pandas as pd
import dash_leaflet as dl
from dash import dcc, html

# Load shapefiles and CSV
study_area = gpd.read_file("C:/Users/admin/Downloads/WAMSI/Shapefile/sat_boundary_MultiPolygon.shp")
study_area1 = gpd.read_file("C:/Users/admin/Downloads/WAMSI/Shapefile/Zone_11/zones_11_MultiPolygon.shp")
study_area2 = gpd.read_file("C:/Users/admin/Downloads/WAMSI/Shapefile/Zone_11/Polygons_MultiPolygon.shp")
points_df = pd.read_csv("C:/Users/admin/Downloads/WAMSI/points.csv")

# Convert GeoDataFrames to GeoJSON for Leaflet
study_area_geojson = dl.GeoJSON(data=study_area.__geo_interface__, id="study-area")
study_area1_geojson = dl.GeoJSON(data=study_area1.__geo_interface__, id="study-area1")
study_area2_geojson = dl.GeoJSON(data=study_area2.__geo_interface__, id="study-area2")

In [3]:
study_area2_geojson

GeoJSON(id='study-area2', data={'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {'Name': 'Polygon_4', 'Id': 0, 'Shape_Leng': 115043.271467, 'Shape_Area': 725649442.348, 'Position': None}, 'geometry': {'type': 'Polygon', 'coordinates': (((115.25012634422886, -32.2189392381792), (115.41383051202676, -32.21793563471107), (115.424778612771, -32.513977053397454), (115.25255939793826, -32.5118924508685), (115.25012634422886, -32.2189392381792)),)}, 'bbox': (115.25012634422886, -32.513977053397454, 115.424778612771, -32.21793563471107)}, {'id': '1', 'type': 'Feature', 'properties': {'Name': 'Polygon_5', 'Id': 0, 'Shape_Leng': 87574.4163417, 'Shape_Area': 449565309.702, 'Position': None}, 'geometry': {'type': 'Polygon', 'coordinates': (((115.4214950557102, -32.51393730816194), (115.42213709242351, -32.51427263322651), (115.42278278528008, -32.514602927942114), (115.4234320794826, -32.514928163578965), (115.42408491843722, -32.515248312164346), (115.424741

In [12]:
import os
shapefiles = [
    "assets/shapefile/Polygons_1_MultiPolygon.shp",
    "assets/shapefile/Polygons_2_MultiPolygon.shp",
    "assets/shapefile/Polygons_3_MultiPolygon.shp",
    "assets/shapefile/Polygons_4_MultiPolygon.shp",
    "assets/shapefile/Polygons_5_MultiPolygon.shp",
    "assets/shapefile/Polygons_6_MultiPolygon.shp"
]

# Convert shapefiles to GeoJSON
geojson_data = {}
for shapefile in shapefiles:
    try:
        gdf = gpd.read_file(shapefile)
        geojson_data[os.path.basename(shapefile)] = gdf.__geo_interface__
    except Exception as e:
        print(f"Error reading {shapefile}: {e}")

In [13]:
geojson_data

{'Polygons_1_MultiPolygon.shp': {'type': 'FeatureCollection',
  'features': [{'id': '0',
    'type': 'Feature',
    'properties': {'Name': 'Polygon_1',
     'Id': 0,
     'Shape_Leng': 64873.9442888,
     'Shape_Area': 236571782.501,
     'Position': None},
    'geometry': {'type': 'Polygon',
     'coordinates': (((115.73257445926502, -31.78672464970537),
       (115.63023311437732, -31.79807946314196),
       (115.57456629757962, -31.647306033750162),
       (115.67727572356176, -31.64004518313432),
       (115.684956102747, -31.652468020394178),
       (115.69071638780964, -31.676655096479397),
       (115.69916480590157, -31.686785648385612),
       (115.70461787612021, -31.700888149734574),
       (115.7056009645194, -31.710924583300574),
       (115.70855023061517, -31.71803306688136),
       (115.71862688917703, -31.728694769919557),
       (115.72573864646552, -31.753124606387477),
       (115.72367696078616, -31.75476812299221),
       (115.72361253271566, -31.7575620350955),
 